<a href="https://colab.research.google.com/github/swoo-nam/project_final_team1/blob/main/%EC%9D%B4%EC%86%8C%ED%98%84_%EC%84%9C%EB%B9%84%EC%8A%A42_%EB%AA%A8%EB%8D%B8%EB%A7%812.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
import numpy as np
import torch
from tqdm.auto import tqdm
import random
import os
import re

def reset_seeds(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

DATA_PATH = "/content/drive/MyDrive/Final_Project/data/"
SEED = 42

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

# csv 불러오기

In [4]:
df = pd.read_csv(f"{DATA_PATH}0904.csv")
df

,keybert_reviews,corrected_reply,store,food,sentiment
0,"[('배달시간도', 0.7589), ('실물이', 0.6541), ('많아요', 0...","이렇게 저희 매장 찾아주셔서 감사드립니다! 맛뿐만 아니라 양도, 서비스도 모두 최상...",NaN,,1
1,"[('시킬게요', 0.8031), ('리뷰가', 0.7633), ('늦었네요', 0...",좋은 리뷰 덕분에 기분이 행복해지는 것 같아요 고객님께서 맛있다고 말씀 주시니 피로...,NaN,,1
2,"[('펩시콜라는', 0.7283), ('주문에', 0.7013), ('다만', 0....",소중한 식사 시간에 저희 매장을 찾아 주셔서 감사드립니다. 저희를 찾아 주신 만큼 ...,NaN,,1
3,"[('말이', 0.7511), ('메뉴', 0.7032), ('없는', 0.4661)]",수많은 매장들을 헤치고 와주셔서 감사합니다! 저희 매장의 메뉴를 제대로 즐기신 거 ...,NaN,,1
4,"[('맛있어요', 0.9021), ('양도많구좋음', 0.9021)]",수많은 매장들을 헤치고 와주셔서 감사합니다 메뉴 그 자체로 행복을 드리고 싶었는데 ...,NaN,,1
...,...,...,...,...,...
4995,"[('맛있게', 0.5977), ('먹었습니다', 0.5977)]",저팔계 중국 마라탕 덮밥 요리입니다. 저희 음식을 맛있게 드시고 소중한 사진 리뷰...,"마라,저팔계 ,저팔계,저팔",덮밥,1
4996,"[('맛있어요', 0.5788), ('먹었어요', 0.5788)]",저팔계 중국 마라탕 덮밥 요리입니다. 저희 음식을 맛있게 드시고 소중한 사진 리뷰...,"마라,저팔계 ,저팔계,저팔",덮밥,0
4997,"[('너무너무', 0.9261), ('맛있다', 0.4438)]",저팔계 중국 마라탕 덮밥 요리입니다. 저희 음식을 맛있게 드시고 소중한 사진 리뷰...,"마라,저팔계 ,저팔계,저팔",덮밥,1
4998,"[('맛있게', 0.5977), ('먹었습니다', 0.5977)]",저팔계 중국 마라탕 덮밥 요리입니다. 저희 음식을 맛있게 드시고 소중한 사진 리뷰...,"마라,저팔계 ,저팔계,저팔",덮밥,1


In [5]:
df["corrected_reply"].isnull().sum()

23

In [6]:
df[df.corrected_reply.isna()]

,keybert_reviews,corrected_reply,store,food,sentiment
625,"[('용기로', 0.6164), ('전부다', 0.5185), ('국그릇만', 0....",NaN,"우시,고기,우시장",,0
636,"[('배달도', 0.7027), ('적어요', 0.5956), ('많이', 0.47...",NaN,"우시,고기,우시장",,0
658,"[('배달음식점은', 0.3868), ('4000원', 0.0927), ('같는',...",NaN,NaN,,0
732,"[('운영', 0.6028), ('시켜', 0.4168), ('늦지만', 0.1899)]",NaN,"우시,고기,우시장",,1
736,"[('배달은', 0.6323), ('너무', 0.5234), ('늦은감이', 0.4...",NaN,"우시,고기,우시장",,1
742,"[('얼큰수육국밥에', 0.8956), ('들어가서', 0.4539), ('많이',...",NaN,"우시,고기,우시장",,0
745,"[('한참지난시간인데도', 0.5925), ('주문이', 0.3591), ('갔습니...",NaN,"우시,고기,우시장",,1
757,"[('간한건지는', 0.4105), ('msg를', 0.1658), ('같은데', ...",NaN,고기,,0
777,"[('시켜먹을', 0.6595), ('자주', 0.2678), ('같습니다', 0....",NaN,"우시,고기,우시장",,1
780,"[('저녁시간보다', 0.6727), ('만큼', 0.5516), ('같습니다', ...",NaN,"우시,고기,우시장",,0


In [7]:
df = df.dropna(subset=['corrected_reply'])

In [8]:
df.columns = ["고객리뷰","사장댓글",'store','food', 'sentiment']
df

,고객리뷰,사장댓글,store,food,sentiment
0,"[('배달시간도', 0.7589), ('실물이', 0.6541), ('많아요', 0...","이렇게 저희 매장 찾아주셔서 감사드립니다! 맛뿐만 아니라 양도, 서비스도 모두 최상...",NaN,,1
1,"[('시킬게요', 0.8031), ('리뷰가', 0.7633), ('늦었네요', 0...",좋은 리뷰 덕분에 기분이 행복해지는 것 같아요 고객님께서 맛있다고 말씀 주시니 피로...,NaN,,1
2,"[('펩시콜라는', 0.7283), ('주문에', 0.7013), ('다만', 0....",소중한 식사 시간에 저희 매장을 찾아 주셔서 감사드립니다. 저희를 찾아 주신 만큼 ...,NaN,,1
3,"[('말이', 0.7511), ('메뉴', 0.7032), ('없는', 0.4661)]",수많은 매장들을 헤치고 와주셔서 감사합니다! 저희 매장의 메뉴를 제대로 즐기신 거 ...,NaN,,1
4,"[('맛있어요', 0.9021), ('양도많구좋음', 0.9021)]",수많은 매장들을 헤치고 와주셔서 감사합니다 메뉴 그 자체로 행복을 드리고 싶었는데 ...,NaN,,1
...,...,...,...,...,...
4995,"[('맛있게', 0.5977), ('먹었습니다', 0.5977)]",저팔계 중국 마라탕 덮밥 요리입니다. 저희 음식을 맛있게 드시고 소중한 사진 리뷰...,"마라,저팔계 ,저팔계,저팔",덮밥,1
4996,"[('맛있어요', 0.5788), ('먹었어요', 0.5788)]",저팔계 중국 마라탕 덮밥 요리입니다. 저희 음식을 맛있게 드시고 소중한 사진 리뷰...,"마라,저팔계 ,저팔계,저팔",덮밥,0
4997,"[('너무너무', 0.9261), ('맛있다', 0.4438)]",저팔계 중국 마라탕 덮밥 요리입니다. 저희 음식을 맛있게 드시고 소중한 사진 리뷰...,"마라,저팔계 ,저팔계,저팔",덮밥,1
4998,"[('맛있게', 0.5977), ('먹었습니다', 0.5977)]",저팔계 중국 마라탕 덮밥 요리입니다. 저희 음식을 맛있게 드시고 소중한 사진 리뷰...,"마라,저팔계 ,저팔계,저팔",덮밥,1


In [9]:
df['store'] = df['store'].fillna("")

<ipython-input-9-2838f85e0d25>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['store'] = df['store'].fillna("")


In [10]:
df = df.sample(frac=1).reset_index(drop=True) # 데이터 랜덤 섞기
df

,고객리뷰,사장댓글,store,food,sentiment
0,"[('술먹고', 0.8712), ('필수', 0.7025), ('진짜', 0.3629)]",맛있는 음식은 물론 최고의 서비스로 좋은 기억만 남겨드리겠습니다! 늘 고객님의 의견...,,,1
1,"[('신경써', 0.7285), ('요청사항', 0.7148), ('감사해요', 0...",안녕하세요 점 리뷰 너무 감사합니다 배달의 쌀국수 은 늘 푸짐한 양과 맛으로 고...,"플러,플러스,배달의쌀",국수,1
2,"[('배달도', 0.8856), ('맛있어요', 0.5147), ('빠르고', 0....",안녕하세요~ 맛있게 드셔주신 고객님 덕분에 기분이 하늘에서 내려올 생각을 안 하네요...,,,1
3,"[('주문해드린건데', 0.7199), ('만족', 0.6033), ('곳인데요',...","아이 정말요~? 감사해요, 저희도 너무 기쁘고 보람차네요 다른 분들께도 저희 사 소...",,,1
4,"[('주문할게요', 0.7941), ('우리아이', 0.5062), ('잘먹었습니다...",안녕하세요 우리 고객님 리뷰와 극찬 진심으로 감사합니다 아이도 너무 이뻐요 배달의...,"플러,플러스",국수,1
...,...,...,...,...,...
4972,"[('오뎅이네요', 0.7064), ('고급스런', 0.6249), ('맛있게', ...","어? 저희에서 특별히 준비한 어묵인데 마으메 드셨나 봐요, 저희 어묵은 시중에서 쉽...",,순대,1
4973,"[('맛잇게', 0.9021), ('먹엇습니당', 0.9021)]",지나가는 한마디에도 큰 힘이 되는 요즘 고객님이 남겨주신 리뷰에 큰 힘이 됩니다 힘...,,,0
4974,"[('우동국물도', 0.706), ('감사합니다', 0.6015), ('맛있어서',...",감사합니다 늘 맛있는 음식들로 손님들의 소중한 한 끼에 다가가겠습니다! 믿고 또 찾...,,,1
4975,"[('감사합당', 0.972), ('좋아요', 0.4377)]",안녕하세요! 주문해 주셔서 감사드립니다! 저희 워커스 베스트 메뉴를 전부 섭렵하셨네...,"워커,워커스",,1


In [152]:
# lst = [('또먹고싶음', 0.9021), ('맛있러료', 0.9021)]

In [153]:
# t = []
# for i in lst:
#     tmp = []
#     tmp.append(i[0])
#     t.extend(tmp)
# t

['또먹고싶음', '맛있러료']

In [156]:
# lst = df['고객리뷰'][0]
# lst

"[('크리스피해서', 0.8349), ('배달', 0.5329), ('빨라요', 0.494)]"

In [11]:
pat = re.compile("[^가-힣 ]")

keyword = []
for lst in df['고객리뷰']:
    tmp = []
    # print(lst)
    # print(type(lst))
    t = pat.sub("",lst)
    t = t.replace("  ",",")
    tmp.append(t)
    keyword.extend(tmp)

keyword

['술먹고,필수,진짜 ',
 '신경써,요청사항,감사해요 ',
 '배달도,맛있어요,빠르고 ',
 '주문해드린건데,만족,곳인데요 ',
 '주문할게요,우리아이,잘먹었습니다 ',
 '여전히,속세의,귀여운 ',
 '시켜서,오랜만에,많고 ',
 '점심,해결한,같네요 ',
 '맛있게,먹었습니다 ',
 '맛있게,먹었습니다 ',
 '먹습니다,고기,맛있어요 ',
 '생각보다,많고,맛있어요 ',
 '맛있게,먹었습니다 ',
 '별로,맛은,매운맛이라고 ',
 '엄청,고기,넉넉하고맛있어요 ',
 '너무맛잇습니다 ',
 '신선하고,,주문했는데 ',
 '분걸려서,누눅하게,별로였지만 ',
 '먹어볼게용,이틀,맛있어서 ',
 '맛있게,잘먹었습니다 ',
 '소세지도,리뷰,맛있습니다 ',
 '맛나게,잘먹었습니다 ',
 '언제나,맛있어요 ',
 '복숭아에이드,짱짱맛입니다 ',
 '맛있게,먹었습니다 ',
 '주시면,김치,감사하겠습니다 ',
 '맛있게,먹었습니다 ',
 '너무,맛있게,먹었어요 ',
 '맛있게,먹었습니다 ',
 '사랑합니다,명이서,배터지게 ',
 '주말저녁타임이라,걸릴걸,굳굳굳 ',
 '맛나게,먹었습니다 ',
 '비냉에,육수도,많고 ',
 '감사합니다,여기는,좋아요 ',
 '집이에요,해장에도,매번맛있게 ',
 '수고하세요,너무,맛있게 ',
 '대만족입니다,항상 ',
 '갈릭디핑소스,추가가,맛있었어요 ',
 '김치,먹었습니다,볶음밥 ',
 '먹으면,국밥집,믿고 ',
 '먹어보세요,피자집,최고입니다 ',
 '친절합니다,기본에,깔끔합니다 ',
 '마늘소스기가막히더라구요 ',
 '냄새안나고깔끔하고맛있어요 ',
 '',
 '와우,깍두기가,엄청맛있네요 ',
 '먹었습니다,좋아요 ',
 '푸짐하고,따봉,맛도 ',
 '스며들어,특히,같아요 ',
 '먹은,며칠,됐는데 ',
 '삼겹은,먹어보고,먹었습니다 ',
 '만족합니다,전부터,같았는데 ',
 '간장이랑,참기름,넣어뒀다가 ',
 '맛있게,잘먹었습니다 ',
 '특별한건,군만두가,맛도 ',
 '감사합니다,잘먹고,있어요 ',
 '시킵니다,여기에서만,닭강정은

In [12]:
df['keyword'] = keyword
df

,고객리뷰,사장댓글,store,food,sentiment,keyword
0,"[('술먹고', 0.8712), ('필수', 0.7025), ('진짜', 0.3629)]",맛있는 음식은 물론 최고의 서비스로 좋은 기억만 남겨드리겠습니다! 늘 고객님의 의견...,,,1,"술먹고,필수,진짜"
1,"[('신경써', 0.7285), ('요청사항', 0.7148), ('감사해요', 0...",안녕하세요 점 리뷰 너무 감사합니다 배달의 쌀국수 은 늘 푸짐한 양과 맛으로 고...,"플러,플러스,배달의쌀",국수,1,"신경써,요청사항,감사해요"
2,"[('배달도', 0.8856), ('맛있어요', 0.5147), ('빠르고', 0....",안녕하세요~ 맛있게 드셔주신 고객님 덕분에 기분이 하늘에서 내려올 생각을 안 하네요...,,,1,"배달도,맛있어요,빠르고"
3,"[('주문해드린건데', 0.7199), ('만족', 0.6033), ('곳인데요',...","아이 정말요~? 감사해요, 저희도 너무 기쁘고 보람차네요 다른 분들께도 저희 사 소...",,,1,"주문해드린건데,만족,곳인데요"
4,"[('주문할게요', 0.7941), ('우리아이', 0.5062), ('잘먹었습니다...",안녕하세요 우리 고객님 리뷰와 극찬 진심으로 감사합니다 아이도 너무 이뻐요 배달의...,"플러,플러스",국수,1,"주문할게요,우리아이,잘먹었습니다"
...,...,...,...,...,...,...
4972,"[('오뎅이네요', 0.7064), ('고급스런', 0.6249), ('맛있게', ...","어? 저희에서 특별히 준비한 어묵인데 마으메 드셨나 봐요, 저희 어묵은 시중에서 쉽...",,순대,1,"오뎅이네요,고급스런,맛있게"
4973,"[('맛잇게', 0.9021), ('먹엇습니당', 0.9021)]",지나가는 한마디에도 큰 힘이 되는 요즘 고객님이 남겨주신 리뷰에 큰 힘이 됩니다 힘...,,,0,"맛잇게,먹엇습니당"
4974,"[('우동국물도', 0.706), ('감사합니다', 0.6015), ('맛있어서',...",감사합니다 늘 맛있는 음식들로 손님들의 소중한 한 끼에 다가가겠습니다! 믿고 또 찾...,,,1,"우동국물도,감사합니다,맛있어서"
4975,"[('감사합당', 0.972), ('좋아요', 0.4377)]",안녕하세요! 주문해 주셔서 감사드립니다! 저희 워커스 베스트 메뉴를 전부 섭렵하셨네...,"워커,워커스",,1,"감사합당,좋아요"


# 모델 불러오기

In [4]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 37.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 43.2 MB/s eta 0:00:00


In [5]:
model_name = "skt/kogpt2-base-v2"

In [6]:
# AutoModelForCausalLM 인과 관련 사용
from transformers import AutoTokenizer, AutoModelForCausalLM

In [7]:
model = AutoModelForCausalLM.from_pretrained(model_name)

In [8]:
tokenizer = AutoTokenizer.from_pretrained(model_name,
                                          bos_token='</s>',
                                          eos_token='</s>',
                                          unk_token='<unk>',
                                          pad_token='<pad>',
                                          mask_token='<mask>',
                                          max_len=1024)

# 데이터셋 만들기

In [18]:
df.head()

,고객리뷰,사장댓글,store,food,sentiment,keyword
0,"[('술먹고', 0.8712), ('필수', 0.7025), ('진짜', 0.3629)]",맛있는 음식은 물론 최고의 서비스로 좋은 기억만 남겨드리겠습니다! 늘 고객님의 의견...,,,1,"술먹고,필수,진짜"
1,"[('신경써', 0.7285), ('요청사항', 0.7148), ('감사해요', 0...",안녕하세요 점 리뷰 너무 감사합니다 배달의 쌀국수 은 늘 푸짐한 양과 맛으로 고...,"플러,플러스,배달의쌀",국수,1,"신경써,요청사항,감사해요"
2,"[('배달도', 0.8856), ('맛있어요', 0.5147), ('빠르고', 0....",안녕하세요~ 맛있게 드셔주신 고객님 덕분에 기분이 하늘에서 내려올 생각을 안 하네요...,,,1,"배달도,맛있어요,빠르고"
3,"[('주문해드린건데', 0.7199), ('만족', 0.6033), ('곳인데요',...","아이 정말요~? 감사해요, 저희도 너무 기쁘고 보람차네요 다른 분들께도 저희 사 소...",,,1,"주문해드린건데,만족,곳인데요"
4,"[('주문할게요', 0.7941), ('우리아이', 0.5062), ('잘먹었습니다...",안녕하세요 우리 고객님 리뷰와 극찬 진심으로 감사합니다 아이도 너무 이뻐요 배달의...,"플러,플러스",국수,1,"주문할게요,우리아이,잘먹었습니다"


In [19]:
class ChatDataset(torch.utils.data.Dataset):
    def __init__(self, df):
        self.df = df

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        item = self.df.iloc[idx]
        sentiment = item["sentiment"]
        store = item["store"]
        food = item["food"]
        review = item["keyword"]
        answer = item["사장댓글"]

        return {
            "sentiment": sentiment,
            "store": store,
            "food": food,
            "review": review,
            "answer": answer
        }

In [20]:
def collate_fn(batch):
    sentiment = torch.tensor([item["sentiment"] for item in batch])
    texts = [
        f"<w>{item['store']}<s><f>{item['food']}<s><q>{item['review']}<s><a>{item['answer']}"
        for item in batch
    ]
    x = tokenizer(texts, return_tensors="pt", padding=True)

    return {"x": x, "sentiment": sentiment}

In [21]:
dt = ChatDataset(df)
dt[0]

{'sentiment': 1,
 'store': '',
 'food': ' ',
 'review': '술먹고,필수,진짜 ',
 'answer': '맛있는 음식은 물론 최고의 서비스로 좋은 기억만 남겨드리겠습니다! 늘 고객님의 의견에 귀 기울이고 고객님의 입장에서 생각하는 매장이 될게요 요청사항 있으시거나 아쉬운 부분 있으셨다면 언제든 매장으로 연락 부탁드리겠습니다! 좋은 하루 보내세요 고객님 '}

In [22]:
dt = ChatDataset(df)
dl = torch.utils.data.DataLoader(dt,batch_size=2,collate_fn=collate_fn)
batch = next(iter(dl))
batch

{'x': {'input_ids': tensor([[ 9724,   461,   405,     0,  9724,   444,   405,   739,     0,  9724,
            455,   405,  7851,  7513,  9075,  8697, 10334,  8265,  8276,   739,
              0,  9724,   439,   405,  7496, 11608,  9330, 11187, 10769, 13222,
          25169, 10400, 10586, 14337,  7489, 18525, 12854,  6872, 16913,  7182,
            376, 10061, 12298, 19495, 11955,  8022,  9504, 42811, 12298, 19495,
          16883, 21660,  9215,  9711, 10341,  6866,  8084, 11544, 10182,  9019,
           8133,  7888,  9314, 30939,  8094,  9679,  9019,  8133,  7816, 10846,
          12668,  7283,  9215, 10275, 16920, 19581, 12854,  6872, 16913,  7182,
            376, 10586, 12557, 11219,  7801,  8084, 12298,  7177,   739,     3,
              3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
              3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
              3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
              3,     

In [23]:
model(**batch["x"]).logits.shape

torch.Size([2, 125, 51200])

# 학습루프

In [24]:
def train_loop(dataloader, model, loss_fn, optimizer, device):
    epoch_loss = 0
    model.train()
    for batch in tqdm(dataloader):
        x = batch["x"].to(device)
        sentiment = batch["sentiment"].to(device)

        pred = model(**x).logits  # batch, seq, 단어별 실수값
        n_class = pred.shape[-1]  # 클래스 개수
        sentiment_pred = pred[:, -1, :]

        pred = pred[:, :-1, :]  # eos 토큰 제외
        pred = pred.reshape(-1, n_class)  # 2차원 형태로 변환

        tgt = x["input_ids"][:, 1:]
        tgt = tgt.flatten()  # 1차원 벡터 형태로 바뀜

        mask = tgt != tokenizer.pad_token_id  # 패딩토큰 제외 -> 손실을 계산할 때 제외
        tgt = tgt[mask]
        pred = pred[mask]

        loss = loss_fn(pred, tgt)

        sentiment_loss = loss_fn(sentiment_pred, sentiment)

        optimizer.zero_grad()
        total_loss = loss + sentiment_loss
        total_loss.backward()
        optimizer.step()

        epoch_loss += total_loss.item()

    epoch_loss /= len(dataloader)

    return epoch_loss

In [25]:
reset_seeds(SEED)

model_name = "skt/kogpt2-base-v2"
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=3e-5)
loss_fn = torch.nn.CrossEntropyLoss()

batch_size = 2
epochs = 10

train_dt = ChatDataset(df)
train_dl = torch.utils.data.DataLoader(train_dt,batch_size=batch_size,shuffle=True,collate_fn=collate_fn)

for i in range(epochs):
    train_loss = train_loop(train_dl, model, loss_fn, optimizer, device)
    print(train_loss)

  0%|          | 0/2489 [00:00<?, ?it/s]

1.9795501781855782


  0%|          | 0/2489 [00:00<?, ?it/s]

1.3492206198149486


  0%|          | 0/2489 [00:00<?, ?it/s]

...

0.6796006445169401


  0%|          | 0/2489 [00:00<?, ?it/s]

0.6378213185270707


  0%|          | 0/2489 [00:00<?, ?it/s]

0.6046618723836995


- 0905_csv 5000개 epochs = 10, 0.6046

# 학습모델 저장

In [26]:
model.save_pretrained(f"{DATA_PATH}0905_kogpt2_chat")

# 저장한 모델 로드

In [9]:
loaded_model = AutoModelForCausalLM.from_pretrained(f"{DATA_PATH}0905_kogpt2_chat").to(device)

In [10]:
!pip install keybert

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 913.0 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 16.3 MB/s eta 0:00:00
  Created wheel for keybert: filename=keybert-0.7.0-py3-none-any.whl size=23765 sha256=9fd2db3eba89bd3992261862c87d58ca533ee93726016380da0d223e2889624a
  Stored in directory: /root/.cache/pip/wheels/66/8d/e6/b0e2f8d883b0fd51819226f67ad9843e04913ce4a97241ff4b
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=fdad6cff59f09ca2dbd96d2d4c33b5f34a80797288e922de4170d506c55d4b1c
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built keybert sentence-transformers


In [11]:
from keybert import KeyBERT

kw_model = KeyBERT()

In [20]:
text = "먹다가 이물질이 나왔어요"

In [21]:
result = kw_model.extract_keywords(text,
                            keyphrase_ngram_range=(1, 1),
                            use_mmr=True,
                            diversity=0.7,
                            stop_words=None,
                            top_n=3)

In [26]:
tmp = []
for i in result:
    tmp.append(i[0])

tmp

['이물질이', '먹다가', '나왔어요']

In [38]:
@torch.no_grad()
def chatbot(model, tokenizer, max_len, device):
    model.eval()
    while True:
        text = input("user > ").strip()
        if text == "quit":
            break

        result = kw_model.extract_keywords(text,
                              keyphrase_ngram_range=(1, 1),
                              use_mmr=True,
                              diversity=0.7,
                              stop_words=None,
                              top_n=3)
        tmp = []
        for i in result:
            tmp.append(i[0])
        keyword = listToString(tmp) # 리스트를 문자열로 변환

        text = "<q>" + keyword + "</s><a>"
        x = tokenizer.encode(text, return_tensors="pt").to(device)
        q_len = len(text) + 1
        result_ids = model.generate(x,
                                    max_length=max_len,
                                    repetition_penalty=2.0,
                                    use_cache=True,
                                    do_sample=True,
                                    temperature=0.8,
                                    top_k=10,
                                    )
        generated_text = tokenizer.decode(result_ids[0])

        generated_text = generated_text[q_len:-4]

        sentences = re.split(r'(?<=[.!?]) +', generated_text)
        if len(sentences) > 3:
            generated_text = ' '.join(sentences[:3])
        else:
            generated_text = ' '.join(sentences)

        print("bot >", generated_text)
        print("--------------------------------------------------------------------------------------------")


In [36]:
def listToString(str_list):
    result = ""
    for s in str_list:
        result += s + ","
    return result.strip()

In [37]:
listToString(tmp)

'이물질이,먹다가,나왔어요,'

In [40]:
chatbot(loaded_model,tokenizer,128,device)

user > 맛이 없어요
bot > 고객님 ~! 먼저 죄송합니다. 부족한 부분으로 실망시켜  사과드리며 늘 저희를 사랑해 주시는 고객님이시군요 정성스러운 사진 리뷰와도 너무 감사한데, 먹음직스럽게 찍어주신 소중한 말씀까지.
--------------------------------------------------------------------------------------------
user > 맛은 있는데 배달이 식어서 왔어요.
bot > 소중한 리뷰 감사드립니다 저희 매장은 맛과 양, 서비스 등 모든 면에서 최고가 되기 위해 항상 최선을 다하고 있습니다, 언제든지 부족함 없이 드실 수 있도록 준비해 보내드리겠습니다.! 앞으로도 만족만 드릴 만반의 준비를 하여 계속 마음속에 남아 있고 싶거든요 <s>~ 좋은 하루 되세요. 고객님 ~
--------------------------------------------------------------------------------------------
user > 먹다가 이물질이 나왔어요ㅠㅠ
bot > 오늘 하루를 시작하며 고객님의 리뷰을 보고 웃음 짓게 됩니다 다음에도 점심 식사를 기분 좋게 드실 수 있길 바랍니다. 저희는 항상 맛있는 음식 제공에 힘내며 언제라도 생각나는 맛집이 되도록 노력하겠습니다, 앞으로도 더욱 만족스러운 식사하실 땐 더 큰 기쁨으로 인사드리겠습니다.
--------------------------------------------------------------------------------------------
user > 가격에 비해 양이 너무 적어서 아쉽네요,,,
bot > 맛있게 드셨다니 다행입니다! 찾아주시는 피드백은 언제든 잘 수렴하고 이를 통해 나날이 발전하는 곳이 되도록 하겠습니다, 더욱더 좋은 모습으로 찾아가겠습니다.<s>. 기쁨 가득한 하루 보내세요 <s>, 감사해요.<s><s><s><s><s> 소중한 리뷰에 제가 느낀 이 행복만큼 고객님도 항상 노력하시길 바라며 오늘